In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import math

# Pose Detection Class
class poseDetector():
    def __init__(self, mode=False, complexity=1, smooth_landmarks=True,
                 enable_segmentation=False, smooth_segmentation=True,
                 detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.complexity = complexity
        self.smooth_landmarks = smooth_landmarks
        self.enable_segmentation = enable_segmentation
        self.smooth_segmentation = smooth_segmentation
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(self.mode, self.complexity, self.smooth_landmarks,
                                     self.enable_segmentation, self.smooth_segmentation,
                                     self.detectionCon, self.trackCon)

    def findPose(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)

        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks,
                                           self.mpPose.POSE_CONNECTIONS)
        return img

    def findPosition(self, img, draw=True):
        self.lmList = []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        return self.lmList

    def findAngle(self, img, p1, p2, p3, draw=True):
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        x3, y3 = self.lmList[p3][1:]

        angle = math.degrees(math.atan2(y3 - y2, x3 - x2) -
                             math.atan2(y1 - y2, x1 - x2))
        if angle < 0:
            angle += 360
            if angle > 180:
                angle = 360 - angle
        elif angle > 180:
            angle = 360 - angle

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 3)
            cv2.line(img, (x3, y3), (x2, y2), (255, 255, 255), 3)

            cv2.circle(img, (x1, y1), 5, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x1, y1), 15, (0, 0, 255), 2)
            cv2.circle(img, (x2, y2), 5, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (0, 0, 255), 2)
            cv2.circle(img, (x3, y3), 5, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x3, y3), 15, (0, 0, 255), 2)

            cv2.putText(img, str(int(angle)), (x2 - 50, y2 + 50),
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        return angle


In [3]:
def trackCurls():
    cap = cv2.VideoCapture(0)
    detector = poseDetector()

    left_count = 0
    right_count = 0
    left_direction = 0
    right_direction = 0
    feedback_left = "LOWER YOUR ARM"
    feedback_right = "LOWER YOUR ARM"

    while cap.isOpened():
        ret, img = cap.read()
        if not ret:
            break

        img = detector.findPose(img, False)
        lmList = detector.findPosition(img, False)

        if len(lmList) != 0:
            # Left Arm
            left_elbow = detector.findAngle(img, 11, 13, 15)
            left_shoulder = detector.findAngle(img, 13, 11, 23)
            left_per = np.interp(left_elbow, (40, 160), (100, 0))
            left_bar = np.interp(left_elbow, (40, 160), (50, 380))

            if left_per == 0 and left_elbow > 160 and left_shoulder < 40:
                feedback_left = "UP"
                if left_direction == 0:
                    left_count += 0.5
                    left_direction = 1
            elif left_per == 100 and left_elbow < 40 and left_shoulder < 40:
                feedback_left = "DOWN"
                if left_direction == 1:
                    left_count += 0.5
                    left_direction = 0

            # Right Arm
            right_elbow = detector.findAngle(img, 12, 14, 16)
            right_shoulder = detector.findAngle(img, 14, 12, 24)
            right_per = np.interp(right_elbow, (40, 160), (100, 0))
            right_bar = np.interp(right_elbow, (40, 160), (50, 380))

            if right_per == 0 and right_elbow > 160 and right_shoulder < 40:
                feedback_right = "UP"
                if right_direction == 0:
                    right_count += 0.5
                    right_direction = 1
            elif right_per == 100 and right_elbow < 40 and right_shoulder < 40:
                feedback_right = "DOWN"
                if right_direction == 1:
                    right_count += 0.5
                    right_direction = 0

            # Visual Feedback
            cv2.rectangle(img, (1080, 50), (1100, 380), (0, 255, 0), 3)
            cv2.rectangle(img, (1080, int(left_bar)), (1100, 380), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, f'{int(left_per)}%', (950, 230), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)

            cv2.rectangle(img, (1180, 50), (1200, 380), (255, 0, 0), 3)
            cv2.rectangle(img, (1180, int(right_bar)), (1200, 380), (255, 0, 0), cv2.FILLED)
            cv2.putText(img, f'{int(right_per)}%', (1250, 230), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)

            # Counters
            cv2.putText(img, f'Left: {int(left_count)}', (10, 50), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)
            cv2.putText(img, f'Right: {int(right_count)}', (10, 100), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)

        cv2.imshow("Curl Tracker", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [5]:
trackCurls()